In [7]:
import sys, os
from omegaconf import OmegaConf
import torch
import numpy as np
import pandas as pd

sys.path.append("/home/jgershon/git/cleo")
from policy_utils import PolicyMPNN, alphabet


test_pdb = "/home/jgershon/projects/itopt/declan/TS1_trp_6_conformers_0008_000_10-atomized-bb-False_4_9_MPNN.pdb"

config = {
    "run_name": "test_v1",
    "output_dir": "/home/jgershon/projects/itopt/policy_mpnn/testing",
    "model_type": "protein_mpnn",
    "N_train": 500,
    "pdb": test_pdb,
    "batch_size": 64,
    "temperature": 1.0,
    "omit_AA": "CX",
    "lr": 1e-4,
    "fixed_residues": "A29 A30 A31 A32 A60 A61 A62 A63 A100 A101 A102 A103",
    "checkpoint_every_n_steps": 10,
    "reward": {
        "_target_": "reward_utils.EnrichAminoAcidReward",
        "AA_to_enrich": "E"
    },
    "eval":False,
}

config = OmegaConf.create(config)

policy_mpnn = PolicyMPNN(config)


In [8]:
# to train policy
policy_mpnn.train()

Training:   0%|          | 2/500 [00:04<19:47,  2.39s/it]

Checkpointing model at step 1 with reward 0.0562


Training:   1%|          | 3/500 [00:07<19:34,  2.36s/it]

Checkpointing model at step 2 with reward 0.0635


Training:   1%|          | 6/500 [00:13<18:05,  2.20s/it]

Checkpointing model at step 5 with reward 0.0686


Training:   2%|▏         | 11/500 [00:23<17:00,  2.09s/it]

Checkpointing model at step 10 with reward 0.0891


Training:  34%|███▎      | 168/500 [05:35<11:03,  2.00s/it]


KeyboardInterrupt: 

In [5]:
path = "/home/jgershon/projects/itopt/policy_mpnn/testing/test_v1_train_metrics.csv"
df = pd.read_csv(path)

ParserError: Error tokenizing data. C error: Expected 3 fields in line 47, saw 4


In [4]:
df

NameError: name 'df' is not defined

In [7]:
500*50

25000

In [14]:
for b in batched_reward:
    print(b)

tensor(124.)
tensor(124.)
